In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import uproot
import sys
import math
import os

dir = "/Users/alexanderantonakis/Desktop/Software/AFrameAnalysis/Macros/"

sys.path.append("../Utils")
sys.path.append("../Configs")
sys.path.append("../DAQ")
sys.path.append("../Mappings")
sys.path.append("../BiasFiles")

from frame_to_crt_fcl import *


daq_df = pd.DataFrame(frame_map, columns = ["Frame", "Frame_FEB", "DAQ_FEB", "Wall"])

all_bias_files = [f for f in os.listdir("../BiasFiles") if os.path.isfile("../BiasFiles/"+f)]
print("All bias settings available")
print(all_bias_files)

daq_df[:4]

In [ ]:
goal = 95 # eff goal
bias_files = []

for f in all_bias_files:
    g = f.split(".root")[0].split("_")[-1]
    if int(g) == goal:
        bias_files.append(f)
    
print("bias files to be examined")
print(bias_files)

In [ ]:
# Function for converting a voltage to a setting number
def calc_setting(v):
    return 255 - ((255.0/4)*(68.1 - v))

In [ ]:
def make_new_fcl(name, settings, avg):
    new_lines = []
    std_fcl = "../DAQ/east_rates_nominal/feb018.fcl"
    count = 0
    with open(std_fcl, 'r') as file:
        # Loop through each line in the file
        for line in file:
        # Strip the newline character and split the line by whitespace
            temp_line = line.strip().split()
            if len(temp_line) > 0:
                if temp_line[0] == "[":
                    #print(temp_line)
                    i = temp_line.index('170,')
                    new_line = temp_line
                    #new_line[i] = "180,"
                    if settings[count] > 230:
                        new_line[i] = "230,"
                    elif 0 < settings[count] < 140:
                        new_line[i] = "140,"
                    elif settings[count] < 0:
                        new_line[i] = str(avg)+","
                    else:
                        new_line[i] = str(settings[count]) + ","
                    count += 1
                    #new_lines.append(new_line)
                    final_line = ""
                    for w in new_line:
                        final_line += w
                        final_line += " "
                    final_line += "\n"
                    new_lines.append(final_line)
                else:
                    new_lines.append(line)
            else:
                new_lines.append(line)
    
    # Open new file in write mode
    with open("../DAQ/FEB_FCL_"+str(goal)+"/"+name, 'w') as file:
        # Loop through each line in the list
        for line in new_lines:
            # Write the line to the file
            file.write(line)
        

In [ ]:
print("loop over bias files")
df = 0
for f in bias_files:
    frame = int(f.split("frame")[1][0])
    print("Current Frame", frame)

    file = uproot.open("../BiasFiles/"+f)  # Replace with your ROOT file
    tree = file["bias_settings_frame"+str(frame)]    
    # Convert the TTree to a pandas DataFrame
    df = tree.arrays(library="pd")
    if int(df['frame'].values[0]) != frame:
        print("Houston we have a problem !!!!")
        break
    df['S'] = df['v'].apply(calc_setting)
    febs = list(set(list(df['feb'].values)))

    # loop over the febs in this bias file ...
    for feb in febs:
        new_feb = str(int(feb))
        daq_feb = daq_df.query("Frame == @frame and Frame_FEB == "+new_feb)["DAQ_FEB"].values[0]
        fcl_file = "feb"+daq_feb+".fcl"
        print("current fcl file", fcl_file)

        # Loop over the number of Module Channels --> # of SiPMs
        settings = []
        for c in range(32):
            strip = int(c /2)
            #print("Channel", c, "strip", strip)
            S = df.query("feb == "+str(feb)+" and strip == "+str(strip))['S'].values[0]
            #print("S", S)
            settings.append(int(math.ceil(S)))
        print("Settings", settings)
        print("")
        avg = np.mean(np.array(list(df.query("v > -1 and S > 140 and feb == "+str(feb))["S"].values)))
        avg = int(math.ceil(avg))
        make_new_fcl(fcl_file, settings, avg)
        

    
df[:2]